## fastq2vcf
### Tutorial intended to cover the analysis of NGS human data. In particular the processing of fastq files in order to get variants in VCF format. 


#### Fastq Downloading

We are going to use public data from the Simons Diversity Project. Specifically 10 selected genomes from different populations. Here below a table with metadata downloaded from ENA repository.

In [38]:
import pandas as pd

projectID = 'PRJEB9586'
! wget -O {projectID}.metadata 'http://www.ebi.ac.uk/ena/data/warehouse/filereport?accession=PRJEB9586&result=read_run&download=txt'

file = pd.read_csv(projectID + '.metadata', sep='\t')
file.head()


,study_accession,secondary_study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,submission_accession,tax_id,scientific_name,instrument_platform,...,sra_aspera,sra_galaxy,cram_index_ftp,cram_index_aspera,cram_index_galaxy,sample_alias,broker_name,sample_title,nominal_sdev,first_created
0,PRJEB9586,ERP010710,SAMEA3302753,ERS1042136,ERX1097974,ERR1019034,ERA486110,9606,Homo sapiens,ILLUMINA,...,fasp.sra.ebi.ac.uk:/vol1/err/ERR101/004/ERR101...,ftp.sra.ebi.ac.uk/vol1/err/ERR101/004/ERR1019034,NaN,NaN,NaN,SAMEA3302753,NaN,simons_data_sample_155,0,2015-09-18
1,PRJEB9586,ERP010710,SAMEA3302890,ERS1042137,ERX1097975,ERR1019035,ERA486110,9606,Homo sapiens,ILLUMINA,...,fasp.sra.ebi.ac.uk:/vol1/err/ERR101/005/ERR101...,ftp.sra.ebi.ac.uk/vol1/err/ERR101/005/ERR1019035,NaN,NaN,NaN,SAMEA3302890,NaN,simons_data_sample_224,0,2015-09-18
2,PRJEB9586,ERP010710,SAMEA3302855,ERS1042138,ERX1097976,ERR1019036,ERA486110,9606,Homo sapiens,ILLUMINA,...,fasp.sra.ebi.ac.uk:/vol1/err/ERR101/006/ERR101...,ftp.sra.ebi.ac.uk/vol1/err/ERR101/006/ERR1019036,NaN,NaN,NaN,SAMEA3302855,NaN,simons_data_sample_180,0,2015-09-18
3,PRJEB9586,ERP010710,SAMEA3302840,ERS1042139,ERX1097977,ERR1019037,ERA486110,9606,Homo sapiens,ILLUMINA,...,NaN,NaN,NaN,NaN,NaN,SAMEA3302840,NaN,simons_data_sample_265,0,2015-09-18
4,PRJEB9586,ERP010710,SAMEA3302867,ERS1042140,ERX1097978,ERR1019038,ERA486110,9606,Homo sapiens,ILLUMINA,...,fasp.sra.ebi.ac.uk:/vol1/err/ERR101/008/ERR101...,ftp.sra.ebi.ac.uk/vol1/err/ERR101/008/ERR1019038,NaN,NaN,NaN,SAMEA3302867,NaN,simons_data_sample_207,0,2015-09-18


EX2. Peer review the second article; submit it via web form 

EX3. Use the following [directed graph](https://courses.cs.ut.ee/MTAT.03.238/2019_fall/uploads/Main/directed_graph.png). Convert it into adjacency matrix representation and then convert the matrix into the graph where the original edges are replaced by "3-hop" edges or in other words - connect only vertices that are reachable in 3 steps of the original graph. 

![image 01](https://courses.cs.ut.ee/MTAT.03.238/2019_fall/uploads/Main/directed_graph.png "Graph")



EX4. Make a transitive closure of the same graph - using three approaches. 1) G*G*G...; 2) making use of the following property - G2 = G*G, G4 = G2*G2, G8 = G4*G4 and 3) with Warshall algorithm. Verify that you got the same answer. 

EX5. Implement a "random walk" procedure following links randomly (equal probabilities). Estimate the probability of being in any given node by varying how you deal with "dead ends" (e.g. node B) and "nodes with no links into it" (e.g. nodes D or A). First, try "re-appearing anywhere" from dead ends only. Secondly, when walking, with 20% probability jump to any random vertex in the network, and 80% of times select randomly and go to one of the neighbouring vertices. For both scenarios, provide the respective probabilities and identify the most important nodes. 

EX6 (Bonus 2p). Use the random walk data from the previous exercise. Implement the "same" random walk procedure but now clearly using matrix multiplications much like in finding transitive closures. Assign uneven probabilities to links - e.g. 80%-20% for two-link nodes (larger probability to alphabetically the first neighbour) and 60%-30%-10% for three link ones. Feel free to modify the input graph a bit. Introduce enough links so that you do not have "dead ends" or nodes where nothing leads into. E.g. I recommend introducing links G->H, B->K, K->H. But feel free to experiment. Of course, you should also figure out what happens to graph multiplications when you have not yet changed the graph. Your task is to report transition probabilities after N steps. How do you know when to stop multiplying? 

EX7 (Bonus 2p). Extend your solution from EX6 and implement the Google Page Rank algorithm (here is a small [playground](https://learnforeverlearn.com/pagerank/)), the algorithm itself is very simple) where 20% of times you would re-appear anywhere in graph. Both using equal-probabilities links and the uneven 80-20, 60-30-10 probabilities. Report the outcome probabilities and compare to results in EX6. 